# 说明

在开发环境中将目录中的 `model目录/V版本`，`notebook目录` 进行 Sync Obs 同步，之后在下面代码里修改同步模型的版本号。


- `model目录/V版本` 中可能含有多个版本，只需要选择需要调试的版本目录进行同步即可

- `notebook目录` 中含 `detect_image.py文件` 和 `detect_video.py文件` 是便于自己本机运行和在开发环境 `Terminal - TensorFlow-2.1.0` 中执行。

在 `Terminal` 中使用命令 `source /home/ma-user/anaconda3/bin/activate TensorFlow-2.1.0` 可以切换到 `TensorFlow-2.1.0` 的环境方便运行


## 查看同步的模型目录和调试所需文件

In [ ]:
!ls -lh model/V0002/model

!ls -lh notebook

## 图片识别 `detect_image.py` 文件

In [ ]:
!python notebook/detect_image.py --image notebook/test.jpg --show_mask True --mask_alpha 0.6 --version V0002

## 视频识别 `detect_video.py` 文件

In [ ]:
!python notebook/detect_video.py --video notebook/test.mp4 --show_mask True --mask_alpha 0.6 --version V0002

## 下面是便于观察的ipynb调试

----

### 开始

detect_image.py文件 和 detect_video.py文件，所需模型依赖。

**参数配置**

- version 模型对应版本


In [ ]:
import os, sys
import tensorflow as tf
from moxing.framework import file
import cv2
import time

%matplotlib inline
import matplotlib.pyplot as plt 

# 参数配置
version = 'V0002' # 模型版本

# 执行所在路径， V0xxx 表示模型版本号
source_path = os.path.join(os.getcwd(), "model/" + version + "/model")
sys.path.append(source_path)

from core.semantic import load_ade20k_model, image_preporcess, image_segment

# 载入模型
model = load_ade20k_model(os.path.join(source_path, 'deeplabv3_xception65_ade20k.h5'))

### 图片识别

detect_image.py 文件内部代码

**参数配置**

- image 用于识别图文件
- show_mask 图片语义掩膜
- mask_alpha 掩膜透明度 0.1 - 0.9

In [ ]:
# 参数配置
image = cv2.imread('notebook/test.jpg') # 用于识别图文件
show_mask = True # 图片语义掩膜
mask_alpha = 0.5 # 掩膜透明度 0.1 - 0.9

# obs桶路径
obs_path = "obs://puddings/ma-ade20k/notebook/out/image"

# 输出目录
out_path = "notebook/out/image"

# 输出目录存在需要删除里边的内容
if os.path.exists(out_path):
    file.remove(out_path, recursive=True)
os.makedirs(out_path)

prev_time = time.time()

# 图片预处理
resized_image, pad = image_preporcess(image)
# 识别并绘制
predict = model.predict(resized_image)
segment = image_segment(predict, pad, image.shape[:2])

# 是否用语义掩膜覆盖
if show_mask:
    cv2.addWeighted(segment, mask_alpha, image, 1-mask_alpha,0, image)
else:
    image = segment
    
# 绘制时间
curr_time = time.time()
exec_time = curr_time - prev_time
print("识别耗时: %.2f ms" %(1000*exec_time))

# 绘制保存
cv2.imwrite(out_path + "/output" + str(round(prev_time * 1000)) + ".jpg", image) 

# 复制保存到桶
print("输出目录：" + out_path)
file.copy_parallel(out_path, obs_path)

# 总图绘制显示ipynb
result = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
plt.figure(figsize=(10,10))
plt.imshow(result)
plt.axis('on')
plt.show()


### 视频识别

detect_video.py 文件内部代码

**参数配置**

- video 视频文件 
- show_mask 图片语义掩膜
- mask_alpha 掩膜透明度 0.1 - 0.9


In [ ]:
# 参数配置
video = cv2.VideoCapture('notebook/test.mp4') # 用于识别的视频文件 
show_mask = True # 图片语义掩膜
mask_alpha = 0.5 # 掩膜透明度 0.1 - 0.9

# 输出目录
out_path = "notebook/out/video"

# 输出目录存在需要删除里边的内容
if os.path.exists(out_path):
    file.remove(out_path, recursive=True)
os.makedirs(out_path)

# 帧数，用于通过帧数取图
frameNum = 0

# obs桶路径
obs_path = "obs://puddings/ma-ade20k/notebook/out/video"

# 输出保存视频
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = video.get(cv2.CAP_PROP_FPS)
size = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
video_out = cv2.VideoWriter(out_path + "/outputVideo.mp4", fourcc, fps, size)

# 视频是否可以打开，进行逐帧识别绘制
while video.isOpened:
    # 视频读取图片帧
    retval, frame = video.read()
    if retval:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    else:
        # 读取失败、结束后释放所有内容
        video.release()
        video_out.release()
        print("没有图像！尝试使用其他视频")
        break

    print('识别帧：%d/%d' % (frameNum, video.get(7)))
    prev_time = time.time()
    
    # 图片预处理
    resized_image, pad = image_preporcess(frame)
    # 识别并绘制
    predict = model.predict(resized_image)
    segment = image_segment(predict, pad, frame.shape[:2])

    # 是否用语义掩膜覆盖
    if show_mask:
        cv2.addWeighted(segment, mask_alpha, frame, 1-mask_alpha,0, frame)
    else:
        frame = segment

    # 绘制时间
    curr_time = time.time()
    exec_time = curr_time - prev_time
    print("识别耗时: %.2f ms" %(1000*exec_time))

    # 视频输出保存
    result = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    video_out.write(result)
    
    # 每300帧取图进行分割保存
    if(frameNum % 300 == 0):
        cv2.imwrite(os.path.join(out_path, str(frameNum) + ".jpg"), result)
    frameNum += 1
    
    # 复制保存到桶
    file.copy_parallel(out_path, obs_path)

    # 绘制结果ipynb显示
    plt.figure(figsize=(10,10))
    plt.imshow(frame)
    plt.axis('on')
    plt.show()

# 保存统计总数并复制保存到桶
print("输出目录：" + out_path)
# 复制保存到桶
file.copy_parallel(out_path, obs_path)

# 任务完成后释放所有内容
video.release()
video_out.release()
